# Proyecto: 

**Alumno**: Jesús Martín Gaytán Villarreal

## Introducción

El Consejo Nacional de Población (CONAPO) del gobierno mexicano, realiza cada 5 años un análisis sobre un estudio multidimensional de la pobreza y establece un índice de marginación.

En https://www.gob.mx/cms/uploads/attachment/file/685354/Nota_te_cnica_IMEyM_2020.pdf se puede consultar el documento de como se calculó el índice para la encuesta del 2020. El índice, por municipio, se puede consultar en :

https://www.gob.mx/conapo/documentos/indices-de-marginacion-2020-284372

Junto con los indicadores más importantes para la medición multidimensional de la pobreza.

Para este proyecto te pedimos lo siguiente:

 1. Un repositorio en github para el proyecto. El repositorio deberá llevar un archivo readme.md en el que se explique lo que se hace y los archivos de código, pero no la base de datos. 
 
 
 2. En el archivo (o los archivos) de jupyter realiza lo siguiente:
     1. Descargar y Leer el archivo de Base de Datos por Municipio 2020 del índice de marginación desde la página de descargas del gobierno federal mexicano en un dataframe. Los datos se encuentran en la pestaña "IMM 2020".
     
    2. Mostrar la descripción del DataFrame (medias, máximos mínimos, etc...) mostrart algunos hallazgos interesantes de la simple inspección visual y por estadísticas básicas de los datos.
    
    3. Realizar una gráfica que permita ver el porcentaje de municipios por estado con índices de marginación "muy bajo", "bajo", "medio", "alto" y "muy alto". Guarda la gráfica en archivo png
    
    4. Realizar una gráfica que muestre el porcentaje de la población, respecto a la población total de cada estado, con índices de marginación "muy bajo", "bajo", "medio", "alto" y "muy alto". Guarda la gráfica en formato jpg
    
    5. ¿Hay coincidencias entra la gráficas anteriores?  ¿Algún hallazgo? Comenta tu análisis.
    
    6. Grafica la relación de porcentaje de analfabetismo respecto al porcentaje de poblaciones en localidades de menos de 5,000 habitantes.
    
    7. ¿Existe una relación? ¿Cómo podrías analizar con que variable tiene mñas corelación el porcentaje de analfabetismo en personas mayores de 15 años?
    
    8. Desarrolla un nuevo DataFrame con indicadores interesantes por estado que se obtengan de los datos a nivel municipal y que pudieran ser de importancia a la hora de definir políticas públicas. Se va a calificar la originalidad de los indicadores. Justifica las decisiones que tomes, y guarda el nuevo dataframe en formato parquet.

In [1]:
# Importamos algunas liberias que utilizaremos
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Comenzamos importando los datos que estaremos analisando desde el archivo .xls. Como se menciona, estos se encuentran en la hoja *IMM 2020*.

In [2]:
marginacion_df = pd.read_excel('./IMM_2020.xls',sheet_name='IMM_2020')

In [28]:
marginacion_df.head()

,CVE_ENT,NOM_ENT,CVE_MUN,NOM_MUN,POB_TOT,ANALF,SBASC,OVSDE,OVSEE,OVSAE,OVPT,VHAC,PL.5000,PO2SM,IM_2020,GM_2020,IMN_2020
0,1,Aguascalientes,1001,Aguascalientes,948990,1.644738,20.367220,0.104799,0.113169,0.378610,0.591434,10.339530,7.523683,54.226594,60.318795,Muy bajo,0.944508
1,1,Aguascalientes,1002,Asientos,51536,3.526405,33.906364,2.650373,0.486448,0.858160,1.352430,22.942305,78.221049,78.565471,56.546071,Muy bajo,0.885433
2,1,Aguascalientes,1003,Calvillo,58250,4.491509,42.482450,0.365177,0.516760,0.800978,1.040411,19.219858,51.301288,79.259777,57.058251,Muy bajo,0.893453
3,1,Aguascalientes,1004,Cosío,17000,3.144867,27.696745,0.712855,0.577354,0.659833,1.030989,22.716866,65.470588,81.726369,57.114030,Muy bajo,0.894326
4,1,Aguascalientes,1005,Jesús María,129929,2.380588,26.692477,0.277034,0.354957,0.860426,1.312652,16.404575,37.164143,56.748753,59.011762,Muy bajo,0.924042


Al inspeccionar la dataframe generada, vemos que los nombres de cada columna son claves de los valores que en realidad representan. Esto puede generar problemas al momento de interpretar su valor. 

Para poder interpretarlos más facilmente, podemos utilizar el diccionario de datos disponible en la hoja "Diccionario" dentro del mismo archivo de Excel. Sin embargo, esta esta hoja incluye tambien un titulo en la primera fila, y la informacion que nos interesa abarca el rango A3:B20 (17 filas), por lo que debemos tomar esto en cuenta al momento de importar el diccionario.

In [15]:
data_dict = pd.read_excel('./IMM_2020.xls',sheet_name='Diccionario',skiprows=2,nrows=17,index_col=0)

Ahora que hemos importado el diccionario de datos como una dataframe, podemos mostrarlo dentro de nuestro cuaderno para mayor facilidad de consulta, en lugar de consultar repetidamente el archivo Excel.

Pero antes, ya que algunos variables tienen una descripcion muy extensa, debemos ajustar la candidad de caracteres mostrados en cada columna, para poder mostrar la descripcion completa

In [26]:
pd.options.display.max_colwidth = 100 

In [27]:
data_dict

,Descripción
Variable,
CVE_ENT,Clave de entidad federativa
NOM_ENT,Nombre de entidad federativa
CVE_MUN,Clave del municipio
NOM_MUN,Nombre del municipio
POB_TOT,Población total
ANALF,Porcentaje de población analfabeta de 15 años o más
SBASC,Porcentaje de población de 15 años o más sin educación básica
OVSDE,Porcentaje de ocupantes en viviendas particulares habitadas sin drenaje ni excusado
OVSEE,Porcentaje de ocupantes en viviendas particulares habitadas sin energía eléctrica
